In [ ]:
import pandas as pd

# Load datasets
textline_data = pd.read_excel("C:/Users/dhanu/Downloads/line_gt_window1 6.xlsx")
ground_truth_data = pd.read_csv("C:/Users/dhanu/Downloads/bof_histograms_chunks 1.csv")

# Display the first few rows to understand the structure
print("Textline Data:")
print(textline_data.head())

print("\nGround Truth Data (BOF histograms):")
print(ground_truth_data.head())


Textline Data:
  Label                             image name
0     െ  MaI12_Page100_line_1.jpg_window_0.jpg
1   തെ   MaI12_Page100_line_1.jpg_window_1.jpg
2     അ  MaI12_Page100_line_1.jpg_window_2.jpg
3  ത്ത   MaI12_Page100_line_1.jpg_window_3.jpg
4   താ   MaI12_Page100_line_1.jpg_window_4.jpg

Ground Truth Data (BOF histograms):
                                    Image    Word_0    Word_1    Word_2  \
0   MaI12_Page100_line_1.jpg_window_0.jpg  0.000000  0.000000  0.046512   
1   MaI12_Page100_line_1.jpg_window_1.jpg  0.000000  0.000000  0.018868   
2  MaI12_Page100_line_1.jpg_window_10.jpg  0.000000  0.011905  0.000000   
3  MaI12_Page100_line_1.jpg_window_11.jpg  0.000000  0.000000  0.018519   
4  MaI12_Page100_line_1.jpg_window_12.jpg  0.017857  0.000000  0.017857   

     Word_3    Word_4  Word_5    Word_6    Word_7  Word_8  ...   Word_90  \
0  0.000000  0.046512     0.0  0.000000  0.000000     0.0  ...  0.000000   
1  0.000000  0.037736     0.0  0.037736  0.037736     0.0  ... 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

# Load datasets
textline_data = pd.read_excel('C:/Users/dhanu/Downloads/line_gt_window1 6.xlsx')
ground_truth_data = pd.read_csv('C:/Users/dhanu/Downloads/bof_histograms_chunks 1.csv')

# Step 1: Match each row in ground_truth_data with its corresponding label in textline_data
# Extract the label for each image window
merged_data = pd.merge(ground_truth_data, textline_data, left_on='Image', right_on='image name')

# Step 2: Prepare features (observations) and labels for HMM
observations = merged_data.loc[:, 'Word_0':'Word_99'].values  # Extract features
labels = merged_data['Label'].values  # Extract labels in Malayalam

# Step 3: Encode labels into integer values
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Display the results to ensure everything aligns correctly
print("Merged Data Preview:")
print(merged_data[['Image', 'Label']].head())

print("\nEncoded Labels Preview:")
print(labels_encoded[:5])

print("\nObservations Shape:", observations.shape)


Merged Data Preview:
                                    Image  Label
0   MaI12_Page100_line_1.jpg_window_0.jpg      െ
1   MaI12_Page100_line_1.jpg_window_1.jpg    തെ 
2  MaI12_Page100_line_1.jpg_window_10.jpg  ാള ക 
3  MaI12_Page100_line_1.jpg_window_11.jpg      െ
4  MaI12_Page100_line_1.jpg_window_12.jpg      െ

Encoded Labels Preview:
[2280  600 2092 2280 2280]

Observations Shape: (3536, 100)


In [6]:
from hmmlearn import hmm
from sklearn.metrics import accuracy_score
import Levenshtein as lev  # For calculating Character Error Rate

# Step 1: Discretize feature vectors using K-means clustering
n_clusters = 10  # Adjust the number of clusters based on data characteristics
kmeans = KMeans(n_clusters=n_clusters)
observations_discrete = kmeans.fit_predict(observations).reshape(-1, 1)

# Step 2: Set up and train the HMM
n_states = 5  # Number of hidden states, can be tuned
n_iter = 100  # Number of iterations for the Baum-Welch algorithm

model = hmm.MultinomialHMM(n_components=n_states, n_iter=n_iter, tol=1e-5)
model.fit(observations_discrete)  # Train HMM on discretized features

# Step 3: Decode the sequence using Viterbi Algorithm
logprob, predicted_states = model.decode(observations_discrete, algorithm="viterbi")

# Step 4: Map Predicted States to Malayalam Characters
# Use label encoder to map back to original Malayalam characters
state_to_char = {i: label_encoder.inverse_transform([i])[0] for i in range(n_states)}
predicted_chars = ''.join(state_to_char[state] for state in predicted_states)
print("state_to_char",state_to_char)

# Convert true encoded labels back to characters for comparison
true_chars = ''.join(label_encoder.inverse_transform(labels_encoded))
print("true_chars",true_chars)
# Align lengths for evaluation
min_length = min(len(true_chars), len(predicted_chars))
true_chars = true_chars[:min_length]
predicted_chars = predicted_chars[:min_length]

print("predicted_chars",predicted_chars)
# Calculate accuracy and Character Error Rate (CER)
accuracy = accuracy_score(list(true_chars), list(predicted_chars))
cer = lev.distance(true_chars, predicted_chars) / len(true_chars)

# Display results
print("Performance (Accuracy):", accuracy)
print("Character Error Rate (CER):", cer)


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


state_to_char {0: '""', 1: '2വാ ', 2: '?', 3: '?മ ', 4: '\xa0പരിഷ്വ െ'}
true_chars െതെ ാള ക െെഷ മാ ിത്തിത്തിആ ത്ര അെെഠ ക ക ാൽ ൽ ർ ഇഇ ഇ ൊ െത്ത കെ ങ്ക ങ്ക ാ െെനെ ന റ െെജെ താ കറ കട്ര ട്രി ട്രി ട ട്ടു ട്ടു ഠ ുഠ ഠ വ ഹ ഹറ ിിരി രി പ പ റ രു രള െതെ യെ യ യ ാി ാാ""പ പ ക ത ആ ആ ധി ച്ചിച്ച ച്ച ച ി െറെ ത ത ന് തം ന്നി ന്നി റി ട്ടു ട്ടു റ അ അ ളെ ള ഇ ഫ എ െഎ ല്ല ല്ല ഇ ാാെറെ തെ ത റ െനെ െകെ ന്ന നം യ യ ാാഫ ന്നു ര െെെ""""""""ട ട എ ക കു കു യ യ യ റ എ എ ല്ല ല്ല യ െഎ ട മെ മ ടു ടു ടു ളെ െതു തു െെട െട ടന ന ച്ച ന െെപ്പെ പ്പ ല്ലാ ാ െെഴെ ഴ ഴം ച്ചു െലെ പ്പു അ ത തന ന്നു ക ക ൊാഒ ഒ ദി """"""െെകെ ക ങ്കാ കാ ജെ ജ ട്ട െജെ ജ ജ ഗ ഗത്ര കാ ത്ര ഠ യ യ യാ ാാഇ ങ്ങ ങ്ങ ല്ല ല്ല ള്ള ങ്ങ ാങ്ങു ട്ടും ഠ പ പ പരി രി രി ച ച ചൽ ൽ ഉ െർ ളം ള തത െനെ ന്ന ന്നാ ാമ അ അ റ ന്ത ന്ത ത െെെ െണ ണം ാ െആട െഠ മെ മ ാാട ട ട ത ത െഉ മൂ നിക്ര മുര ത്ത ത്ത തര രാ ഠാ ാ െെചെ ച ക്ര ല ാാല്ല ല്ല ഠ ല ല ാാക്ര മ ാന ൻ മ മം ഠ അ അഅ റ നി െൈചെ ച ചി ിിതി തി ർ ച്ച ച്ചി ര ന്തിട്ട ട്ട റ ിി െെമെ മറ മ ന്ന ന്നു ന്തറ നു റ റ റ അ അ അറ ന്ത ന്തക ക കാ ാത രാപു പു പു രി രി ൈെനെ ന റി ിര െക്ക ക

In [7]:
from hmmlearn import hmm
from sklearn.metrics import accuracy_score
import Levenshtein as lev  # For calculating Character Error Rate

# Step 1: Adjusted Discretization with Increased Clusters
n_clusters = 20  # Increased to capture more detail
kmeans = KMeans(n_clusters=n_clusters)
observations_discrete = kmeans.fit_predict(observations).reshape(-1, 1)

# Step 2: Set up and train the HMM with more hidden states and iterations
n_states = 10  # Increased number of hidden states
n_iter = 200  # Increased iterations for better convergence

model = hmm.MultinomialHMM(n_components=n_states, n_iter=n_iter, tol=1e-6)
model.fit(observations_discrete)

# Step 3: Decode the sequence using Viterbi Algorithm
logprob, predicted_states = model.decode(observations_discrete, algorithm="viterbi")

# Step 4: Map Predicted States to Malayalam Characters
state_to_char = {i: label_encoder.inverse_transform([i % len(label_encoder.classes_)])[0] for i in range(n_states)}
predicted_chars = ''.join(state_to_char[state] for state in predicted_states)

# Convert true encoded labels back to characters for comparison
true_chars = ''.join(label_encoder.inverse_transform(labels_encoded))

# Align lengths for evaluation
min_length = min(len(true_chars), len(predicted_chars))
true_chars = true_chars[:min_length]
predicted_chars = predicted_chars[:min_length]

# Calculate accuracy and Character Error Rate (CER)
accuracy = accuracy_score(list(true_chars), list(predicted_chars))
cer = lev.distance(true_chars, predicted_chars) / len(true_chars)

# Display results
print("Performance (Accuracy):", accuracy)
print("Character Error Rate (CER):", cer)


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Performance (Accuracy): 0.06016311166875784
Character Error Rate (CER): 0.8377666248431619


In [37]:
from hmmlearn import hmm
from sklearn.metrics import accuracy_score
import Levenshtein as lev  # For calculating Character Error Rate

# Step 1: Adjusted Discretization with Increased Clusters
n_clusters = 500  # Increased to capture more detail
kmeans = KMeans(n_clusters=n_clusters)
observations_discrete = kmeans.fit_predict(observations).reshape(-1, 1)

# Step 2: Set up and train the HMM with more hidden states and iterations
n_states = 180  # Increased number of hidden states
n_iter = 100  # Increased iterations for better convergence

model = hmm.MultinomialHMM(n_components=n_states, n_iter=n_iter, tol=1e-4)
model.fit(observations_discrete)

# Step 3: Decode the sequence using Viterbi Algorithm
logprob, predicted_states = model.decode(observations_discrete, algorithm="viterbi")

# Step 4: Map Predicted States to Malayalam Characters
state_to_char = {i: label_encoder.inverse_transform([i % len(label_encoder.classes_)])[0] for i in range(n_states)}
predicted_chars = ''.join(state_to_char[state] for state in predicted_states)

# Convert true encoded labels back to characters for comparison
true_chars = ''.join(label_encoder.inverse_transform(labels_encoded))

# Align lengths for evaluation
min_length = min(len(true_chars), len(predicted_chars))
true_chars = true_chars[:min_length]
predicted_chars = predicted_chars[:min_length]

# Calculate accuracy and Character Error Rate (CER)
accuracy = accuracy_score(list(true_chars), list(predicted_chars))
cer = lev.distance(true_chars, predicted_chars) / len(true_chars)

# Display results
print("Performance (Accuracy):", accuracy)
print("Character Error Rate (CER):", cer)


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340
Fitting a model with 32399 free scalar parameters with only 3536 data points will result in a degenerate solution.


Performance (Accuracy): 0.08230865746549561
Character Error Rate (CER): 0.7872020075282309


In [8]:
from hmmlearn import hmm
from sklearn.metrics import accuracy_score
import Levenshtein as lev  # For calculating Character Error Rate

# Step 1: Adjusted Discretization with Increased Clusters
n_clusters = 500  # Increased to capture more detail
kmeans = KMeans(n_clusters=n_clusters)
observations_discrete = kmeans.fit_predict(observations).reshape(-1, 1)

# Step 2: Set up and train the HMM with more hidden states and iterations
n_states = 180  # Increased number of hidden states
n_iter = 100  # Increased iterations for better convergence

model = hmm.MultinomialHMM(n_components=n_states, n_iter=n_iter, tol=1e-4)
model.fit(observations_discrete)

# Step 3: Decode the sequence using Viterbi Algorithm
logprob, predicted_states = model.decode(observations_discrete, algorithm="viterbi")

# Step 4: Map Predicted States to Malayalam Characters
state_to_char = {i: label_encoder.inverse_transform([i % len(label_encoder.classes_)])[0] for i in range(n_states)}
predicted_chars = ''.join(state_to_char[state] for state in predicted_states)
print("state_to_char",state_to_char)
print(predicted_chars)
# Convert true encoded labels back to characters for comparison
true_chars = ''.join(label_encoder.inverse_transform(labels_encoded))

# Align lengths for evaluation
min_length = min(len(true_chars), len(predicted_chars))
true_chars = true_chars[:min_length]
predicted_chars = predicted_chars[:min_length]

# Calculate accuracy and Character Error Rate (CER)
accuracy = accuracy_score(list(true_chars), list(predicted_chars))
cer = lev.distance(true_chars, predicted_chars) / len(true_chars)

# Display results
print("Performance (Accuracy):", accuracy)
print("Character Error Rate (CER):", cer)


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340
Fitting a model with 32399 free scalar parameters with only 3536 data points will result in a degenerate solution.


state_to_char {0: '""', 1: '2വാ ', 2: '?', 3: '?മ ', 4: '\xa0പരിഷ്വ െ', 5: '\xa0പ്രയ ത്നെ റ ', 6: '\xa0ഭൂഷു നീ', 7: '\xa0വിനാ\xa0ന', 8: '\xa0ശ്ശി രഃപു', 9: 'അ', 10: 'അ ', 11: 'അ ടം ', 12: 'അ ട്ര ', 13: 'അ ട്രി ', 14: 'അ തി ', 15: 'അ ഥ', 16: 'അ യ ', 17: 'അ രു ', 18: 'അ റ ', 19: 'അ ഷ ', 20: 'അ െ', 21: 'അ ൽ ', 22: 'അം ', 23: 'അട ', 24: 'അട്ര ', 25: 'അന ', 26: 'അപ്പ ', 27: 'അപ്ര ', 28: 'അഭ ', 29: 'അറ ', 30: 'അറു', 31: 'അറു ', 32: 'അവ', 33: 'അവ ', 34: 'അവി ', 35: 'അവിഘനമ', 36: 'അഹ ', 37: 'ആ', 38: 'ആ ', 39: 'ആ ട്ട ', 40: 'ആ മ ', 41: 'ആ ാ', 42: 'ആം ', 43: 'ആട ', 44: 'ആദി ', 45: 'ആയ്യ ', 46: 'ആരു ', 47: 'ഇ', 48: 'ഇ ', 49: 'ഇ ങ്ങ ', 50: 'ഇ ത ', 51: 'ഇ ഫ ', 52: 'ഇ രി ', 53: 'ഇ റ്റ ', 54: 'ഇ ാ', 55: 'ഇ െ', 56: 'ഉ ', 57: 'ഉ ഉ ', 58: 'ഉ ദൂ നി ', 59: 'ഉ ന ', 60: 'ഉ സ്ര മ ', 61: 'ഉണ്ട ', 62: 'ഉദന്വതാ െ', 63: 'ഉലപ്ര ', 64: 'ഋകീഴ ', 65: 'എ ', 66: 'എ അ ', 67: 'എ ക ', 68: 'എ ച്ചും ', 69: 'എ ട ', 70: 'എ റ ', 71: 'എ ഴ ', 72: 'എ ഹ ', 73: 'എ െ', 74: 'എം ', 75: 'എന്ത ', 76: 'എന്ന ', 77: 'എന്നു', 78: 'എല്ല', 7